In [25]:
import pandas as pd

In [38]:
ctc=pd.read_json('https://drive.google.com/uc?export=download&id=1KECYQy-nuof5xfQUR_ZjzqZgSjdPyngd')

In [39]:
import numpy as np

In [40]:
ctc=ctc.replace(r'^\s*$', np.nan, regex=True)

In [41]:
df_email=ctc.dropna(subset=['Email']).groupby(['Email']).size().sort_values()
m_email=ctc[ctc.Email.isin(df_email[df_email>1].index)].groupby('Email').agg({'Id':'unique'})
df_Phone=ctc.dropna(subset=['Phone']).groupby(['Phone']).size().sort_values()
m_Phone=ctc[ctc.Phone.isin(df_Phone[df_Phone>1].index)].groupby('Phone').agg({'Id':'unique'})
df_OrderId=ctc.dropna(subset=['OrderId']).groupby(['OrderId']).size().sort_values()
m_OrderId=ctc[ctc.OrderId.isin(df_OrderId[df_OrderId>1].index)].groupby('OrderId').agg({'Id':'unique'})

In [42]:
m_email=m_email.reset_index()
m_Phone=m_Phone.reset_index()
m_OrderId=m_OrderId.reset_index()

In [44]:
b=m_email.Id.append(m_Phone.Id).append(m_OrderId.Id).values.tolist()
list_b=[list(x) for x in b]

def pairs(lst):
    i = iter(lst)
    first = prev = item = i.__next__()
    for item in i:
        yield prev, item
        prev = item
    yield item, first

list_b

import networkx
g = networkx.Graph()
for sub_list in list_b:
    for edge in pairs(sub_list):
            g.add_edge(*edge)

list_con=[c for c in sorted(networkx.connected_components(g), key=len, reverse=True)]

<generator object connected_components at 0x7f0aa2bead50>

In [77]:
d_subm={'ticket_id':[],
        'ticket_trace':[],
        'Contacts':[]}

for i in list_con:
  trace=sorted(list(i))
  d_subm['ticket_id'].append(trace[0])
  d_subm['ticket_trace'].append(trace)
  d_subm['Contacts'].append(ctc[ctc.Id.isin(trace)]['Contacts'].sum())

In [78]:
df_submission=pd.DataFrame(d_subm)

In [88]:
df_single=ctc[~ctc.Id.isin(df_submission.ticket_id)][['Id','Contacts']].rename(columns={'Id':'ticket_id'})

In [89]:
df_single['ticket_trace']=df_single.ticket_id

In [93]:
df_finalize=pd.concat([df_submission,df_single]).sort_values('ticket_id').reset_index(drop=True)

In [95]:
ctc

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,NaN,1,NaN
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,NaN,9125983679,0,NaN
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,NaN,300364407,2,NaN
...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


In [94]:
df_finalize

,ticket_id,ticket_trace,Contacts
0,0,0,1
1,1,"[1, 2458, 98519, 115061, 140081, 165605, 476346]",12
2,2,"[2, 159312, 322639, 348955]",4
3,3,3,0
4,4,4,2
...,...,...,...
499995,499995,499995,2
499996,499996,499996,4
499997,499997,499997,2
499998,499998,499998,1
